# Parallel Function Calling

利用できるモデルとAPIバージョンが限られています。
https://learn.microsoft.com/ja-jp/azure/ai-services/openai/how-to/function-calling?tabs=python#parallel-function-calling

In [1]:
!python -m pip install openai
!python -m pip install python-dotenv
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()
deployment = os.environ['DEPLOYMENT']
apikey = os.environ['API_KEY']
base = os.environ['BASE']
api_version=os.environ['API_VERSION']

In [3]:
client = AzureOpenAI(
  azure_endpoint = base, 
  api_key=apikey,  
  api_version=api_version
)

In [4]:
TOOLS = [
    {
        "type":"function",
        "function": {
            "name": "switch_airconditioner",
            "description": "エアコンの電源をスイッチします。",
            "parameters": {
                "type": "object",
                "properties": {
                    "switch": {
                        "type": "boolean",
                        "description": "オンの場合はtrue,オフの場合はfalse"
                    }
                },
                "required": ["switch"]
            }
          }
    },
    {
        "type":"function",
        "function": {
            "name": "switch_lamp",
            "description": "照明のスイッチをオン・オフします。",
            "parameters": {
                "type": "object",
                "properties": {
                    "switch": {
                        "type": "boolean",
                        "description": "オンの場合はtrue,オフの場合はfalse"
                    }
                },
                "required": ["switch"]
            }
          }
    }
]

# 並列機能呼び出しを期待する入力

In [5]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "エアコンと電気をつけてください。"}
    ],
    tools=TOOLS,
    tool_choice="auto",
    temperature=0,
)

if response.choices[0].finish_reason=='tool_calls':
    for tool_call in response.choices[0].message.tool_calls:
        print(tool_call.function)

Function(arguments='{"switch": true}', name='switch_airconditioner')
Function(arguments='{"switch": true}', name='switch_lamp')


# 単一機能呼び出しを期待する入力

In [6]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "エアコンを消してください。"}
    ],
    tools=TOOLS,
    tool_choice="auto",
    temperature=0,
)

if response.choices[0].finish_reason=='tool_calls':
    for tool_call in response.choices[0].message.tool_calls:
        print(tool_call.function)

Function(arguments='{"switch":false}', name='switch_airconditioner')
